# Evaluate Python UDF Performance (Iris)

In [1]:
import os

import numpy as np
import pandas as pd

import json

import duckdb

from pprint import pprint

### Load timing data

In [2]:
timing_dir = 'analyze_outputs'
files = [os.path.abspath(os.path.join(timing_dir, f)) for f in os.listdir(timing_dir) if '.json' in f]
files

['/Users/udeshuk/Developer/mldb/eval/analyze_outputs/opt_vec_scalar_iris3_15000000.json',
 '/Users/udeshuk/Developer/mldb/eval/analyze_outputs/opt_vec_iris4_10000000.json',
 '/Users/udeshuk/Developer/mldb/eval/analyze_outputs/opt_vec3_iris2_5000000.json',
 '/Users/udeshuk/Developer/mldb/eval/analyze_outputs/opt_vec_scalar_iris4_10000000.json',
 '/Users/udeshuk/Developer/mldb/eval/analyze_outputs/opt_vec|w5-d512_iris3_1000000.json',
 '/Users/udeshuk/Developer/mldb/eval/analyze_outputs/opt_vec_scalar_iris5_5000000.json',
 '/Users/udeshuk/Developer/mldb/eval/analyze_outputs/opt_vec|w5-d512_iris3_15000000.json',
 '/Users/udeshuk/Developer/mldb/eval/analyze_outputs/opt_vec_iris5_1000000.json',
 '/Users/udeshuk/Developer/mldb/eval/analyze_outputs/opt_vec3_iris3_15000000.json',
 '/Users/udeshuk/Developer/mldb/eval/analyze_outputs/opt_vec_iris2_1000000.json',
 '/Users/udeshuk/Developer/mldb/eval/analyze_outputs/opt_vec|w3-d128_iris3_10000000.json',
 '/Users/udeshuk/Developer/mldb/eval/analyze_

In [3]:
times_raw = {}

for file in files:
    with open(file, "r") as f:
        name = file.split("/")[-1].split(".")[0]
        workload = name.split("_")[-1]
        key = name.split("_iris")[0]
        run = name.split("_iris")[1][0]
        times_raw[key + "~" + workload + "~" + run] = json.loads(f.read())

In [4]:
times = []

for k, v in times_raw.items():
    t = {}
    t['dataset'] = 'iris'
    t['pipeline'] = k.split("~")[0]
    t['workload'] = k.split("~")[1]
    t['run'] = k.split("~")[2]

    if t['run'] ==  '1':
        prediction = v['children'][0]['children'][0]['children'][0]
    else:
        prediction = v['children'][0]['children'][0]
        # if t['pipeline'] == 'opt_vec2':
        #     print(t['workload'])
        #     pprint(prediction)
    t["predict"] = prediction['timing'] * 1000000
    t["load"] = 0
    move = prediction['children'][0]['children'][0]
    t["move"] = move['timing'] * 1000000
    
    times.append(t)

In [5]:
times_df = pd.DataFrame.from_records(times).loc[:, ['dataset', 'pipeline', 'workload', 'move', 'load', 'predict']]
times_df['workload'] = times_df['workload'].astype(int)
times_df.sort_values(['pipeline', 'workload'])
times_df

,dataset,pipeline,workload,move,load,predict
0,iris,opt_vec_scalar,15000000,1925467.0,0,8679695.0
1,iris,opt_vec,10000000,1060667.0,0,16912116.0
2,iris,opt_vec3,5000000,1041674.0,0,4511019.0
3,iris,opt_vec_scalar,10000000,1048125.0,0,5176771.0
4,iris,opt_vec|w5-d512,1000000,146129.0,0,23001825.0
...,...,...,...,...,...,...
59,iris,opt_vec,10000000,1051651.0,0,16775249.0
60,iris,opt_vec2,5000000,564841.0,0,7739211.0
61,iris,opt_vec3,15000000,2849155.0,0,14521084.0
62,iris,opt,5000000,586376.0,0,252963288.0


In [6]:
stro = pd.read_csv('sub_time_rel_opt.csv')
stro['move'] = stro['move'] + stro['move_rev']
stro = stro.drop(columns=['move_rev'])
stro['dataset'] = 'iris'
stro['pipeline'] = 'opt_vec_V3'

times_df = pd.concat([times_df, stro])

### Write to times table

In [7]:
con = duckdb.connect("../test.db")
tables = con.sql(f"SHOW TABLES").df()
has_table = tables[tables.name == 'times'].shape[0] == 1
    
if not has_table:
    con.sql(f"CREATE TABLE times (dataset varchar, workload integer, move integer, load integer, predict integer, ts timestamp DEFAULT current_timestamp)")

con.sql(f"DELETE FROM times WHERE pipeline LIKE 'opt%'")
con.sql(f"INSERT INTO times (dataset, pipeline, workload, move, load, predict) SELECT * FROM times_df")
con.sql(f"SELECT * FROM times").show()
con.close()

┌─────────┬──────────┬─────────┬───────┬───────────┬─────────────────────────┬───────────────┐
│ dataset │ workload │  move   │ load  │  predict  │           ts            │   pipeline    │
│ varchar │  int32   │  int32  │ int32 │   int32   │        timestamp        │    varchar    │
├─────────┼──────────┼─────────┼───────┼───────────┼─────────────────────────┼───────────────┤
│ iris    │  1000000 │   66407 │ 11404 │    644862 │ 2024-04-16 15:39:06.832 │ duckdb_python │
│ iris    │  1000000 │   69000 │ 14205 │    711004 │ 2024-04-16 15:39:13.889 │ duckdb_python │
│ iris    │  1000000 │   87882 │ 15013 │    714544 │ 2024-04-16 15:39:37.287 │ duckdb_python │
│ iris    │  1000000 │   66269 │ 14332 │    696981 │ 2024-04-16 15:39:42.694 │ duckdb_python │
│ iris    │  1000000 │   75381 │ 14991 │    691012 │ 2024-04-16 15:39:54.801 │ duckdb_python │
│ iris    │  5000000 │  328913 │ 15558 │   5179009 │ 2024-04-16 15:40:13.76  │ duckdb_python │
│ iris    │  5000000 │  349613 │ 15064 │   2955772

In [8]:
con.close()